# 🌟 Star Tracker Simulation with Gaia Subset

This notebook shows how to simulate star tracker functionality using a **filtered Gaia catalog**.

We'll:
- Query a subset of bright stars from the Gaia catalog (Gmag < 8)
- Simulate observed positions with rotation
- Recover the spacecraft attitude by solving Wahba's problem

Requires internet access and the `astroquery` package to run.

In [ ]:
from astroquery.gaia import Gaia
import numpy as np
import pandas as pd
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt

# Query bright stars with good astrometry
query = '''
SELECT TOP 100
  source_id, ra, dec, phot_g_mean_mag
FROM gaiaedr3.gaia_source
WHERE phot_g_mean_mag < 8
AND parallax > 0
AND astrometric_excess_noise < 1
'''

job = Gaia.launch_job_async(query)
gaia_df = job.get_results().to_pandas()
gaia_df.head()

### 🧭 Convert RA/Dec to unit vectors in inertial frame

In [ ]:
def radec_to_unitvec(ra_deg, dec_deg):
    ra = np.radians(ra_deg)
    dec = np.radians(dec_deg)
    x = np.cos(dec) * np.cos(ra)
    y = np.cos(dec) * np.sin(ra)
    z = np.sin(dec)
    return np.vstack([x, y, z]).T

catalog_vecs = radec_to_unitvec(gaia_df['ra'].values, gaia_df['dec'].values)

### 🔄 Simulate spacecraft rotation (attitude)

In [ ]:
# Simulate a 3D rotation
true_rot = R.from_euler('xyz', [10, 5, 25], degrees=True)
observed_vecs = true_rot.apply(catalog_vecs)

### 🧮 Estimate rotation using Wahba's problem

In [ ]:
def solve_attitude(observed, catalog):
    H = observed.T @ catalog
    U, _, Vt = np.linalg.svd(H)
    R_est = Vt.T @ U.T
    return R.from_matrix(R_est)

R_est = solve_attitude(observed_vecs, catalog_vecs)
R_est.as_euler('xyz', degrees=True)

### 📈 Visualize a slice of star directions in RA/Dec

In [ ]:
plt.figure(figsize=(6, 4))
plt.scatter(gaia_df['ra'], gaia_df['dec'], label='Catalog', c='blue')
plt.title("Gaia Star Catalog Subset")
plt.xlabel("RA (deg)")
plt.ylabel("Dec (deg)")
plt.grid(True)
plt.legend()
plt.show()

## ✅ Summary
- Queried a filtered Gaia star list
- Simulated star tracker observations with a known rotation
- Recovered the spacecraft attitude using SVD (Wahba's problem)

This method underpins many real-world spacecraft attitude determination systems.